In [23]:
import sympy as sp
from gurobipy import Model, LinExpr, QuadExpr, GRB

# Define a function to convert SymPy expression to Gurobi expression
def sympy_to_gurobi(sympy_expr, symbol_map, model, aux_var_count=[0]):
    """
    Recursively convert a SymPy expression to a Gurobi expression, 
    handling exponentials and other complex expressions with auxiliary variables and constraints.
    
    Parameters:
        sympy_expr (sp.Expr): SymPy expression to convert.
        symbol_map (dict): Mapping from SymPy symbols to Gurobi variables.
        model (gurobipy.Model): Gurobi model to add constraints for complex expressions.
        aux_var_count (list): A list to keep track of the auxiliary variable count.
        
    Returns:
        Gurobi expression (LinExpr, QuadExpr, or constant).
    """
    if isinstance(sympy_expr, sp.Symbol):
        return symbol_map[sympy_expr]
    
    elif isinstance(sympy_expr, sp.Add):
        return sum(sympy_to_gurobi(arg, symbol_map, model, aux_var_count) for arg in sympy_expr.args)
    
    elif isinstance(sympy_expr, sp.Mul):
        result = 1
        for arg in sympy_expr.args:
            result *= sympy_to_gurobi(arg, symbol_map, model, aux_var_count)
        return result
    
    elif isinstance(sympy_expr, sp.Pow):
        base, exp = sympy_expr.args
        if exp == 2:
            base_expr = sympy_to_gurobi(base, symbol_map, model, aux_var_count)
            return QuadExpr(base_expr * base_expr)
        else:
            raise ValueError("Non-quadratic powers are not supported by Gurobi.")
    
    elif isinstance(sympy_expr, sp.exp):
        # Decompose the argument of the exponential function
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        
        # Introduce an auxiliary variable for the argument expression
        aux_var_name = f"exp_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1  # Increment the counter for unique naming
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg",lb = 0, vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        
        # Create the auxiliary variable for the exponential function
        exp_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrExp(arg_aux_var, exp_aux_var)
        
        return exp_aux_var
    
    elif isinstance(sympy_expr, sp.log):
        # Decompose the argument of the logarithm function
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        
        # Introduce an auxiliary variable for the argument expression
        aux_var_name = f"log_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        
        # Create the auxiliary variable for the logarithm function
        log_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrLog(arg_aux_var, log_aux_var)
    
        return log_aux_var

    elif isinstance(sympy_expr, sp.sin):
        # Decompose the argument of the sine function
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        
        # Introduce an auxiliary variable for the argument expression
        aux_var_name = f"sin_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        
        # Handle sine using a piecewise linear approximation or other methods
        sin_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # You can add piecewise constraints here for approximation
        
        return sin_aux_var

    elif isinstance(sympy_expr, sp.cos):
        # Decompose the argument of the cosine function
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        
        # Introduce an auxiliary variable for the argument expression
        aux_var_name = f"cos_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        
        # Handle cosine using a piecewise linear approximation or other methods
        cos_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # You can add piecewise constraints here for approximation
        
        return cos_aux_var
    elif isinstance(sympy_expr, sp.tan):
        # Decompose the argument of the tangent function
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        
        # Introduce an auxiliary variable for the argument expression
        aux_var_name = f"tan_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        
        # Handle tangent using a piecewise linear approximation or other methods
        tan_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        # You can add piecewise constraints here for approximation
        
        return tan_aux_var



    elif isinstance(sympy_expr, sp.Number):
        return float(sympy_expr)
    
    else:
        raise ValueError(f"Unsupported SymPy expression: {sympy_expr}")

# Example usage
x, y = sp.symbols('x y')
sympy_expr = sp.exp((x + y)/23)
print("Sympy expression:", sympy_expr)

# Create a Gurobi model
model = Model("example")
x_gurobi = model.addVar(name="x", lb=4, vtype=GRB.CONTINUOUS, ub=40)
y_gurobi = model.addVar(name="y", lb=4, vtype=GRB.CONTINUOUS, ub=20)

# Map SymPy symbols to Gurobi variables
symbol_map = {x: x_gurobi, y: y_gurobi}

# Convert SymPy expression to Gurobi expression and update model
gurobi_expr = sympy_to_gurobi(sympy_expr, symbol_map, model)

# Set the objective and optimize
model.setObjective(gurobi_expr, GRB.MAXIMIZE)
model.params.OutputFlag = 0  # Suppress output
model.optimize()

# Output the optimized values
if model.status == GRB.OPTIMAL:
    print(f"Optimal x: {x_gurobi.X}")
    print(f"Optimal y: {y_gurobi.X}")
    print(f"Optimal objective value: {model.objVal}")
print("Sympy evaluated expression:", sympy_expr.subs({x: x_gurobi.X, y: y_gurobi.X}).evalf())


Sympy expression: exp(x/23 + y/23)
Optimal x: 40.0
Optimal y: 20.0
Optimal objective value: 13.5813245225782
Sympy evaluated expression: 13.5813245225782
